# `Headlines POS Pattern Extraction (HPPE)`


`Steps for achieving HPPE:`

- `Reading generated headlines .txt file`
- `Determine POS Tags for all the headlines`
- `POS Tags are fed to removeIdentical() function and return unique POS Tags`
- `Unique POS Tags are matched with POS Tags of original headlines`

In [33]:
import pandas as pd
import numpy as np
import stanza
from tqdm import tqdm, trange
import itertools

In [34]:
stanza.download('ur') # 'en' for english

2023-04-04 12:10:00 INFO: Downloading default packages for language: ur (Urdu)...
2023-04-04 12:10:01 INFO: File exists: /Users/saifhassan/stanza_resources/ur/default.zip.
2023-04-04 12:10:02 INFO: Finished downloading models and saved to /Users/saifhassan/stanza_resources.


In [35]:
# Build a Urdu pipeline, with customized processor list and no logging, and force it to use CPU
print("Building a Urdu pipeline...")
ur_nlp = stanza.Pipeline('ur', processors='tokenize,lemma,pos,depparse')

2023-04-04 12:10:02 INFO: Loading these models for language: ur (Urdu):
| Processor | Package |
-----------------------
| tokenize  | udtb    |
| pos       | udtb    |
| lemma     | udtb    |
| depparse  | udtb    |

2023-04-04 12:10:02 INFO: Use device: cpu
2023-04-04 12:10:02 INFO: Loading: tokenize
2023-04-04 12:10:02 INFO: Loading: pos
2023-04-04 12:10:02 INFO: Loading: lemma


Building a Urdu pipeline...


2023-04-04 12:10:02 INFO: Loading: depparse
2023-04-04 12:10:02 INFO: Done loading processors!


## `Reading generated headlines and converting to csv`

In [48]:
# reading generated headlines and converting to dataframe
file = open('data/news-gen-120.txt', 'r') # picked sample headlines
contents = file.read()
contents = contents.split('\n')
df = pd.DataFrame(contents, columns=['Headline'])
print(len(contents), df.shape)
df.head()
df.to_csv('data/news-gen-120.csv')

501 (501, 1)


## `Pos Tags Extraction for Headlines`

In [49]:
# Processing urdu text for pos tags
#ur_doc = ur_nlp("اسٹاف رپورٹ لاہور ڈاکٹروں کا گھروں سے باہر نکل جانے کیخلاف لاہور ہائیکورٹ میں درخواست دائر کردی گئی پاکستان کرکٹ")

def return_pos_tags(list_headlines):
    pos_tags = []
    sentences = []
    for sen in tqdm(range(0, len(list_headlines)-1)):
        ur_doc = ur_nlp(list_headlines[sen])
        dummy_pos = ""
        dummy_sen = ""
        for i, sent in enumerate(ur_doc.sentences):
            for word in sent.words:
                dummy_pos += word.pos + " "
                dummy_sen += word.text + " "

            pos_tags.append(dummy_pos)
            sentences.append(dummy_sen)
    print(len(pos_tags), len(sentences))
    return pos_tags, sentences

In [67]:
# reading csv file to determine POS Tags
filename = 'data/news-gen-120.csv'
df = pd.read_csv(filename)
df.head()

,Unnamed: 0,Headline
0,0,کراچی اردو پوائنٹ اخبارتازہ ترین این این ئی 08...
1,1,کراچیاردو پوائنٹ اخبار تازہ ترین14دسمبر 2014ءگ...
2,2,لاہوراردو پوائنٹ اخبارتازہ ترین لائن 17 اپریل2...
3,3,لاہوراردو پوائنٹ اخبارتازہ ترین لائن 11 جنوری2...
4,4,قائد اعظم ٹرافی کے تیسرے رانڈ کے چوتھے دن سنٹر...


In [68]:
list_headlines = df['Headline'].tolist()
print(list_headlines[:1])

['کراچی اردو پوائنٹ اخبارتازہ ترین این این ئی 08 نومبر2019ءملکی کرنسی مارکیٹوں میں بدھ کوپاکستانی روپ�']


In [69]:
pos_tags, sentences = return_pos_tags(list_headlines)

100%|██████████| 500/500 [01:09<00:00,  7.20it/s]

500 500


In [70]:
#storing pos tags and headlines in csv file
filename = 'data/pos-tags-gen.csv'
df = pd.DataFrame(list(zip(sentences, pos_tags)), columns=['sentences', 'pos_tags'])
df.to_csv(filename, index=False)

## `Removing identical POS tags`

In [55]:
filename = 'data/pos-tags-gen.csv' # filename --> from which you want to remove identical
df = pd.read_csv(filename)
df.head()

,sentences,pos_tags
0,کراچی اردو پوائنٹ اخبارتازہ ترین این این ئی 08...,PROPN PROPN NOUN ADJ ADJ PROPN PROPN PROPN PRO...
1,کراچیاردو پوائنٹ اخبار تازہ ترین14دسمبر 2014ءگ...,PROPN PROPN NOUN ADJ PROPN PROPN PROPN NOUN AD...
2,لاہوراردو پوائنٹ اخبارتازہ ترین لائن 17 اپریل2...,PROPN PROPN ADJ PROPN NOUN PROPN PROPN ADJ NOU...
3,لاہوراردو پوائنٹ اخبارتازہ ترین لائن 11 جنوری2...,PROPN PROPN PROPN PROPN NOUN PROPN PROPN PROPN...
4,قائد اعظم ٹرافی کے تیسرے رانڈ کے چوتھے دن سنٹر...,NOUN PROPN NOUN ADP ADJ NOUN ADP ADJ NOUN PROP...


In [56]:
#removing duplicates Pos Tags (only contigous) in each record
def removeIdentical(text):
    result = []
    for i in tqdm(text):
        sentence = ' '.join(k for k, _ in itertools.groupby(i[0].split()))
        sentence = sentence
        result.append(sentence)

    return result

In [57]:
pos_tags_list = df['pos_tags'].tolist()

In [58]:
pos_2dlist = np.array(pos_tags_list).reshape(len(pos_tags_list),1)

In [59]:
result_list = removeIdentical(pos_2dlist)
df_unique = pd.DataFrame(result_list, columns=['unique_pos'])
df['unique_pos'] = df_unique
df.to_csv(filename.split('.')[0]+'-unique.csv', index=False)

100%|██████████| 500/500 [00:00<00:00, 97387.94it/s]


## `Matching POS tags for generated headlines with original`

In [75]:
df_org = pd.read_csv('data/pos-tags-org-unique.csv')
df_gen = pd.read_csv('data/pos-tags-gen-unique.csv')
df_org.head()

,sentences,pos_tags,identical_pos
0,عالمی بینک عسکریت پسندی سے متاثرہ خاندانوں کی ...,ADJ NOUN NOUN NOUN ADP ADJ NOUN ADP NOUN VERB ...,ADJ NOUN ADP ADJ NOUN ADP NOUN VERB AUX
1,مالی سال 2020 ریٹرن فائل کرنے والوں کی تعداد م...,ADJ NOUN NUM NOUN NOUN VERB ADP ADP NOUN ADP N...,ADJ NOUN NUM NOUN VERB ADP NOUN ADP NUM NOUN
2,جاپان کو سندھ کے خصوصی اقتصادی زون میں سرمایہ ...,PROPN ADP PROPN ADP ADJ ADJ NOUN ADP ADJ NOUN ...,PROPN ADP PROPN ADP ADJ NOUN ADP ADJ NOUN ADP ...
3,برامدات 767 فیصد بڑھ کر ارب 16 کروڑ ڈالر سے زائد,NOUN NUM NOUN VERB AUX NUM NUM NUM NOUN ADP ADJ,NOUN NUM NOUN VERB AUX NUM NOUN ADP ADJ
4,کے الیکٹرک کو اضافی بجلی گیس کی فراہمی کے قانو...,ADP NOUN ADP ADJ NOUN NOUN ADP NOUN ADP ADJ NO...,ADP NOUN ADP ADJ NOUN ADP NOUN ADP ADJ NOUN AD...


In [76]:
df_gen.head()

,sentences,pos_tags,unique_pos
0,کراچی اردو پوائنٹ اخبارتازہ ترین این این ئی 08...,PROPN PROPN NOUN ADJ ADJ PROPN PROPN PROPN PRO...,PROPN NOUN ADJ PROPN NOUN ADP NOUN ADJ NOUN
1,کراچیاردو پوائنٹ اخبار تازہ ترین14دسمبر 2014ءگ...,PROPN PROPN NOUN ADJ PROPN PROPN PROPN NOUN AD...,PROPN NOUN ADJ PROPN NOUN ADP PROPN NOUN ADP N...
2,لاہوراردو پوائنٹ اخبارتازہ ترین لائن 17 اپریل2...,PROPN PROPN ADJ PROPN NOUN PROPN PROPN ADJ NOU...,PROPN ADJ PROPN NOUN PROPN ADJ NOUN PROPN ADP ...
3,لاہوراردو پوائنٹ اخبارتازہ ترین لائن 11 جنوری2...,PROPN PROPN PROPN PROPN NOUN PROPN PROPN PROPN...,PROPN NOUN PROPN NOUN ADP NOUN VERB
4,قائد اعظم ٹرافی کے تیسرے رانڈ کے چوتھے دن سنٹر...,NOUN PROPN NOUN ADP ADJ NOUN ADP ADJ NOUN PROP...,NOUN PROPN NOUN ADP ADJ NOUN ADP ADJ NOUN PROP...


In [77]:
list_org = df_org['identical_pos'].tolist()
list_gen = df_gen['unique_pos'].tolist()
list_gen_sentences = df_gen['sentences'].tolist()
print(len(list_org), len(list_gen), len(list_gen_sentences))

1038384 500 500


In [78]:
top_25 = df_org['identical_pos'].value_counts()[:25].index.tolist()

In [79]:
# maching generated POS tags with original
list_org_unique = set(list_org)
indices_matched = [i for i, item in enumerate(list_gen) if item in list_org_unique]
len(indices_matched)

92

In [80]:
filtered_headlines = []
for i in indices_matched:
    filtered_headlines.append(list_gen_sentences[i])
len(filtered_headlines)

92

In [81]:
# storing filtered headlines
df_filtered = pd.DataFrame(filtered_headlines, columns=['filtered'])
df_filtered.to_csv("filtered-headlines.csv", index=False)            